In [1]:
import pandas as pd
import wandb

api = wandb.Api()

detectors = ["oriented", "rotated"]
backbones = ["resnet50", "resnet18", "mobilenetv3large", "efficientnet_b0", "efficientnet_b1", "efficientnet_b2", "efficientnet_b3"]
resolutions = ["256", "512", "800"]
run_name = 'pre:0_preb:1_freezebn:0_skipflip:0_skipimgtrf:1_trainlayers:4_lr:0.000100'
header = ["detector", "backbone", "resolution", "valid-mAP", "valid-mIoU", "valid-Precision", "valid-Recall", "valid-F1-Score", "valid-dtheta"]
data = []
for detector in detectors:
    for backbone in backbones:
        for resolution in resolutions:
            entity, project = "crazyboy9103", f"{detector}-{backbone}-detdemo-{resolution}" # "pre:1_preb:1_freezebn:0_skipflip:0_skipimgtrf:1_trainlayers:4_lr:0.000100"
            runs = api.runs(entity + "/" + project)

            metrics = [detector, backbone, resolution]

            for run in runs:
                # .summary contains the output keys/values
                #  for metrics such as accuracy.
                #  We call ._json_dict to omit large files
                if run.name == run_name:
                    metric_dict = run.summary._json_dict
                    try:
                        for metric in header[3:]:
                            metrics.append(metric_dict[metric])
                    except:
                        print(project, run.name)
        
            data.append(metrics)

# Create the DataFrame
df = pd.DataFrame(data, columns=header)
       


# runs_df.to_csv(f"{project}.csv", index=False)

/usr/lib/python3/dist-packages/requests/__init__.py:89: RequestsDependencyWarning: urllib3 (2.0.4) or chardet (3.0.4) doesn't match a supported version!
  warnings.warn("urllib3 ({}) or chardet ({}) doesn't match a supported "


rotated-resnet50-mvtec-256 pre:0_preb:1_freezebn:0_skipflip:0_skipimgtrf:1_trainlayers:4_lr:0.000100


In [15]:
df[(df["detector"] == "oriented") & (df["resolution"] == "800")]

,detector,backbone,resolution,valid-mAP,valid-mIoU,valid-Precision,valid-Recall,valid-F1-Score,valid-dtheta
2,oriented,resnet50,800,0.992422,0.887696,0.997738,0.985933,0.991674,0.858562
5,oriented,resnet18,800,0.978770,0.877411,0.987222,0.956774,0.971375,0.864472
8,oriented,mobilenetv3large,800,0.993974,0.881663,1.000000,0.989771,0.994791,0.871678
11,oriented,efficientnet_b0,800,0.992337,0.887810,0.997976,0.984213,0.990954,0.865893
14,oriented,efficientnet_b1,800,0.992028,0.885814,0.998028,0.985575,0.991690,0.867599
17,oriented,efficientnet_b2,800,0.993849,0.892132,0.993917,0.987906,0.990750,0.870046
20,oriented,efficientnet_b3,800,0.993629,0.892710,1.000000,0.985601,0.992678,0.871035


In [113]:
temp = pd.DataFrame(columns=["detector", "backbone"] + header[3:])
for detector in detectors:
    for backbone in backbones:
        t = df[(df["detector"] == detector) & (df["backbone"] == backbone)][header[3:]].mean().to_frame().T
        t["detector"] = detector
        t["backbone"] = backbone
        temp = pd.concat([temp, t], axis=0)

/tmp/ipykernel_2234344/1913393678.py:7: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  temp = pd.concat([temp, t], axis=0)


In [114]:
temp.to_excel("by_detector_backbone.xlsx", index=False)

In [110]:
temp = pd.DataFrame(columns=["detector", "resolution"] + header[3:])
for detector in detectors:
    for resolution in resolutions:
        t = df[(df["detector"] == detector) & (df["resolution"] == resolution)][header[3:]].mean().to_frame().T
        t["detector"] = detector
        t["resolution"] = resolution
        temp = pd.concat([temp, t], axis=0)

/tmp/ipykernel_2234344/3675455201.py:7: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  temp = pd.concat([temp, t], axis=0)


In [112]:
temp.to_excel("by_detector_resolution.xlsx", index=False)

In [105]:
temp = pd.DataFrame(columns=["backbone"] + header[3:])
for backbone in backbones:
    t = df[(df["backbone"] == backbone)][header[3:]].mean().to_frame().T
    t["backbone"] = backbone
    temp = pd.concat([temp, t], axis=0)

/tmp/ipykernel_2234344/2512922952.py:5: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  temp = pd.concat([temp, t], axis=0)


In [108]:
temp.to_excel("by_backbone.xlsx", index=False)

In [92]:
temp.to_excel("average_metrics.xlsx", index=False)

In [35]:
df[df["run"].str.contains('256')].to_excel("256.xlsx", index=False)

In [36]:
df[df["run"].str.contains('512')].to_excel("512.xlsx", index=False)

In [37]:
df[df["run"].str.contains('800')].to_excel("800.xlsx", index=False)